### 기존 하루 동안의 교통량 => 교통량 증감량으로 변환하기
- 하루 교통량 - 전체기간 교통량 평균

In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from datetime import datetime

In [2]:
traffic_org=pd.read_csv('data/인천교통량_지역추가_수정.csv')
traffic_org=pd.DataFrame(traffic_org)
traffic_org

,date,region_id,daily
0,2021-07-01,2301063.0,30068.0
1,2021-07-01,2301064.0,29077.0
2,2021-07-01,2301066.0,2327.0
3,2021-07-01,2304052.0,282790.0
4,2021-07-01,2304053.0,19515.0
...,...,...,...
10751,2021-12-31,2309070.0,73255.0
10752,2021-12-31,2309071.0,39782.0
10753,2021-12-31,2309072.0,65293.0
10754,2021-12-31,2309073.0,28005.0


In [3]:
traffic_org['date'] = pd.to_datetime(traffic_org['date'])
traffic_org.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10756 entries, 0 to 10755
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   date       10756 non-null  datetime64[ns]
 1   region_id  10756 non-null  float64       
 2   daily      10756 non-null  float64       
dtypes: datetime64[ns](1), float64(2)
memory usage: 252.2 KB


In [4]:
month_avg = traffic_org.set_index('date').groupby([pd.Grouper(freq='1M'),'region_id']).mean().round()
month_avg=month_avg.reset_index()
month_avg

,date,region_id,daily
0,2021-07-31,2301063.0,25117.0
1,2021-07-31,2301064.0,22503.0
2,2021-07-31,2301066.0,2898.0
3,2021-07-31,2304052.0,226639.0
4,2021-07-31,2304053.0,17714.0
...,...,...,...
355,2021-12-31,2309070.0,66595.0
356,2021-12-31,2309071.0,37510.0
357,2021-12-31,2309072.0,60774.0
358,2021-12-31,2309073.0,24445.0


In [5]:
from dateutil.relativedelta import relativedelta

month_avg['date']= month_avg['date'].apply(lambda x : x+relativedelta(days=1))
month_avg

,date,region_id,daily
0,2021-08-01,2301063.0,25117.0
1,2021-08-01,2301064.0,22503.0
2,2021-08-01,2301066.0,2898.0
3,2021-08-01,2304052.0,226639.0
4,2021-08-01,2304053.0,17714.0
...,...,...,...
355,2022-01-01,2309070.0,66595.0
356,2022-01-01,2309071.0,37510.0
357,2022-01-01,2309072.0,60774.0
358,2022-01-01,2309073.0,24445.0


In [6]:
# pd.set_option('display.max_rows', 50)
traff_ex_month= traffic_org

traff_ex_month['month'] = traff_ex_month['date'].apply(lambda x:x.month)

traff_avg_list=pd.DataFrame()

for idx, month in month_avg.iterrows():
    m1 = month['date'].month
    region_id = month['region_id']
    
    month_traff = traff_ex_month[(traff_ex_month['month']==m1)&(traff_ex_month['region_id']==region_id)]
    
    month_traff['traffic_avg']=month['daily']

    traff_avg_list = pd.concat([traff_avg_list, month_traff])
    

traff_avg_list
# traff_ex_month['traffic_diff'] = traff_ex_month.apply(lambda x: x['daily']-x['traffic_avg'],axis=1)
# traff_ex_month = traff_ex_month.query('"2021-08-11"<= date <= "2021-10-26"')
# traff_ex_month

C:\Users\admin\AppData\Local\Temp\ipykernel_11892\1649772447.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  month_traff['traffic_avg']=month['daily']
C:\Users\admin\AppData\Local\Temp\ipykernel_11892\1649772447.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  month_traff['traffic_avg']=month['daily']
C:\Users\admin\AppData\Local\Temp\ipykernel_11892\1649772447.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] 

,date,region_id,daily,month,traffic_avg
3383,2021-08-29,2301063.0,222.0,8,25117.0
3436,2021-08-30,2301063.0,34305.0,8,25117.0
3489,2021-08-31,2301063.0,50161.0,8,25117.0
1944,2021-08-01,2301064.0,4150.0,8,22503.0
1996,2021-08-02,2301064.0,3831.0,8,22503.0
...,...,...,...,...,...
10523,2021-12-27,2309075.0,7778.0,12,22478.0
10581,2021-12-28,2309075.0,23273.0,12,22478.0
10639,2021-12-29,2309075.0,26765.0,12,22478.0
10697,2021-12-30,2309075.0,28600.0,12,22478.0


In [17]:
traff_ex_month=traffic_org
ex_month=[]
traff_ex_month['traff_avg']=0

for idx, traff in traffic_org.iterrows():
    m1=traff['date'].month
    print('index::::',idx)
    if(m1==7):
        ex_month.append(0.0)
        print(m1,0,0)
        continue
    for idx_, avg in month_avg.iterrows():
        m2=avg['date'].month
        if(m1==m2 and traff['region_id']==avg['region_id']):
            ex_month.append(avg['daily']) 
            print(m1, m2, avg['daily'])
            break
    else:
        ex_month.append(0.0) 
            
traff_ex_month['traff_avg']=ex_month
traff_ex_month

index:::: 0
7 0 0
index:::: 1
7 0 0
index:::: 2
7 0 0
index:::: 3
7 0 0
index:::: 4
7 0 0
index:::: 5
7 0 0
index:::: 6
7 0 0
index:::: 7
7 0 0
index:::: 8
7 0 0
index:::: 9
7 0 0
index:::: 10
7 0 0
index:::: 11
7 0 0
index:::: 12
7 0 0
index:::: 13
7 0 0
index:::: 14
7 0 0
index:::: 15
7 0 0
index:::: 16
7 0 0
index:::: 17
7 0 0
index:::: 18
7 0 0
index:::: 19
7 0 0
index:::: 20
7 0 0
index:::: 21
7 0 0
index:::: 22
7 0 0
index:::: 23
7 0 0
index:::: 24
7 0 0
index:::: 25
7 0 0
index:::: 26
7 0 0
index:::: 27
7 0 0
index:::: 28
7 0 0
index:::: 29
7 0 0
index:::: 30
7 0 0
index:::: 31
7 0 0
index:::: 32
7 0 0
index:::: 33
7 0 0
index:::: 34
7 0 0
index:::: 35
7 0 0
index:::: 36
7 0 0
index:::: 37
7 0 0
index:::: 38
7 0 0
index:::: 39
7 0 0
index:::: 40
7 0 0
index:::: 41
7 0 0
index:::: 42
7 0 0
index:::: 43
7 0 0
index:::: 44
7 0 0
index:::: 45
7 0 0
index:::: 46
7 0 0
index:::: 47
7 0 0
index:::: 48
7 0 0
index:::: 49
7 0 0
index:::: 50
7 0 0
index:::: 51
7 0 0
index:::: 52
7 0 0
ind

,date,region_id,daily,month,traff_avg
0,2021-07-01,2301063.0,30068.0,7,0.0
1,2021-07-01,2301064.0,29077.0,7,0.0
2,2021-07-01,2301066.0,2327.0,7,0.0
3,2021-07-01,2304052.0,282790.0,7,0.0
4,2021-07-01,2304053.0,19515.0,7,0.0
...,...,...,...,...,...
10751,2021-12-31,2309070.0,73255.0,12,72746.0
10752,2021-12-31,2309071.0,39782.0,12,39090.0
10753,2021-12-31,2309072.0,65293.0,12,64076.0
10754,2021-12-31,2309073.0,28005.0,12,25540.0


In [22]:
traff_temp_p=traff_ex_month
traff_temp_p = traff_temp_p.query('"2021-08-11"<= date <= "2021-10-26"')
traff_temp_p

,date,region_id,daily,month,traff_avg
2464,2021-08-11,2301064.0,2847.0,8,22503.0
2465,2021-08-11,2301066.0,2601.0,8,2898.0
2466,2021-08-11,2304052.0,195342.0,8,226639.0
2467,2021-08-11,2304053.0,18745.0,8,17714.0
2468,2021-08-11,2304054.0,152212.0,8,131428.0
...,...,...,...,...,...
6801,2021-10-26,2309072.0,68235.0,10,53158.0
6802,2021-10-26,2309073.0,23321.0,10,19707.0
6803,2021-10-26,2309075.0,26107.0,10,16494.0
6804,2021-10-26,2331031.0,11522.0,10,10403.0


In [24]:
traff_diff=[]
for idx, traff in traff_temp_p.iterrows():
    traff_diff.append(traff['daily']-traff['traff_avg'])
traff_temp_p['traff_diff']=traff_diff
traff_temp_p

C:\Users\admin\AppData\Local\Temp\ipykernel_11892\1553409921.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  traff_temp_p['traff_diff']=traff_diff


,date,region_id,daily,month,traff_avg,traff_diff
2464,2021-08-11,2301064.0,2847.0,8,22503.0,-19656.0
2465,2021-08-11,2301066.0,2601.0,8,2898.0,-297.0
2466,2021-08-11,2304052.0,195342.0,8,226639.0,-31297.0
2467,2021-08-11,2304053.0,18745.0,8,17714.0,1031.0
2468,2021-08-11,2304054.0,152212.0,8,131428.0,20784.0
...,...,...,...,...,...,...
6801,2021-10-26,2309072.0,68235.0,10,53158.0,15077.0
6802,2021-10-26,2309073.0,23321.0,10,19707.0,3614.0
6803,2021-10-26,2309075.0,26107.0,10,16494.0,9613.0
6804,2021-10-26,2331031.0,11522.0,10,10403.0,1119.0


In [25]:
traff_std=StandardScaler().fit_transform(traff_temp_p[['traff_diff']])
traff_temp_p['traff_std']=traff_std
traff_temp_p

C:\Users\admin\AppData\Local\Temp\ipykernel_11892\1711758457.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  traff_temp_p['traff_std']=traff_std


,date,region_id,daily,month,traff_avg,traff_diff,traff_std
2464,2021-08-11,2301064.0,2847.0,8,22503.0,-19656.0,-0.700576
2465,2021-08-11,2301066.0,2601.0,8,2898.0,-297.0,0.064259
2466,2021-08-11,2304052.0,195342.0,8,226639.0,-31297.0,-1.160488
2467,2021-08-11,2304053.0,18745.0,8,17714.0,1031.0,0.116726
2468,2021-08-11,2304054.0,152212.0,8,131428.0,20784.0,0.897127
...,...,...,...,...,...,...,...
6801,2021-10-26,2309072.0,68235.0,10,53158.0,15077.0,0.671655
6802,2021-10-26,2309073.0,23321.0,10,19707.0,3614.0,0.218775
6803,2021-10-26,2309075.0,26107.0,10,16494.0,9613.0,0.455783
6804,2021-10-26,2331031.0,11522.0,10,10403.0,1119.0,0.120203


In [26]:
traff_temp_p.describe()

,region_id,daily,month,traff_avg,traff_diff,traff_std
count,4.342000e+03,4342.000000,4342.000000,4342.000000,4342.000000,4.342000e+03
mean,2.307257e+06,40278.438047,9.117918,42201.925610,-1923.487563,-1.984460e-15
std,4.887163e+03,46531.725910,0.775046,43972.672962,25314.256448,1.000115e+00
min,2.301063e+06,0.000000,8.000000,0.000000,-225390.000000,-8.828711e+00
25%,2.305057e+06,8995.500000,9.000000,14095.000000,-5392.250000,-1.370438e-01
50%,2.306071e+06,26035.500000,9.000000,32797.000000,148.500000,8.186005e-02
75%,2.308084e+06,55122.500000,10.000000,52691.000000,6962.500000,3.510674e-01
max,2.331038e+06,285838.000000,10.000000,239222.000000,93967.000000,3.788440e+00


In [ ]:
# region_id 로 group
traff_region=pd.DataFrame(traffic_org.groupby(['region_id'], as_index=False)[['daily']].mean().round(0))
traff_region

In [ ]:
traff_temp=traffic_org
traff_avg=[]
for idx, traff in traffic_org.iterrows():
    for idx_, avg in traff_region.iterrows():
        if(traff['region_id']==avg['region_id']):
            traff_avg.append(avg['daily'])
traff_temp['traff_avg']=traff_avg

In [ ]:
traff_temp

In [ ]:
traff_diff=[]
for idx, traff in traffic_org.iterrows():
    traff_diff.append(traff['daily']-traff['traff_avg'])
traff_temp['traff_diff']=traff_diff
traff_temp

In [ ]:
traff_temp.to_csv('data/인천교통량_변화량_수정.csv',index=False)

In [ ]:
traff_temp_p=traff_temp
for idx, traff in traff_temp.iterrows():
    if(traff['date']<'2021-08-01' or traff['date']>'2021-10-31'):
        traff_temp_p=traff_temp_p.drop(idx)
traff_temp_p

In [ ]:
traff_std=StandardScaler().fit_transform(traff_temp_p[['traff_diff']])
traff_temp_p['traff_std']=traff_std
traff_temp_p

In [ ]:
traff_temp_p.describe()

### 31935
- -191969 ~ -134841
- -134841 ~ -77713
- -77713 ~ -20585
- -20585 ~ 36543
- 36543 ~ 93671

In [27]:
traff_std_no=[]
tmax = traff_temp_p['traff_diff'].max()
tmin = traff_temp_p['traff_diff'].min()
tdff = (tmax-tmin)/10

for idx, traff in traff_temp_p.iterrows():
    no=0
    if(traff['traff_diff']>=(tmax-tdff)):
        no=10
    elif(traff['traff_diff']>=(tmax-tdff*2) and traff['traff_diff']<(tmax-tdff)):
        no=9
    elif(traff['traff_diff']>=0 and traff['traff_diff']<(tmax-tdff*2)):
        no=8
    elif(traff['traff_diff']>=(tmax-tdff*4) and traff['traff_diff']<0):
        no=7
    elif(traff['traff_diff']>=(tmax-tdff*5) and traff['traff_diff']<(tmax-tdff*4)):
        no=6
    elif(traff['traff_diff']>=(tmax-tdff*6) and traff['traff_diff']<(tmax-tdff*5)):
        no=5
    elif(traff['traff_diff']>=(tmax-tdff*7) and traff['traff_diff']<(tmax-tdff*6)):
        no=4
    elif(traff['traff_diff']>=(tmax-tdff*8) and traff['traff_diff']<(tmax-tdff*7)):
        no=3
    elif(traff['traff_diff']>=(tmax-tdff*9) and traff['traff_diff']<(tmax-tdff*8)):
        no=2
    elif(traff['traff_diff']<(tmax-tdff*9)):
        no=1
    
    print(no)
    traff_std_no.append(no)
    
traff_temp_p['std_no']=traff_std_no
traff_temp_p.head()

7
7
7
8
8
8
8
8
7
7
7
7
8
8
8
8
8
9
8
8
8
9
7
8
8
8
7
8
7
8
8
7
7
8
8
8
8
7
7
7
8
7
8
8
7
8
8
8
7
8
7
7
7
7
7
8
8
8
8
8
7
7
7
7
8
8
8
8
8
9
8
8
8
9
7
8
8
8
8
8
7
8
8
7
8
8
8
8
8
8
8
8
8
7
8
8
7
8
7
8
8
8
8
7
7
8
7
8
6
7
6
8
6
7
6
6
7
7
8
7
7
6
7
7
7
8
8
8
7
8
8
8
7
8
8
7
8
8
7
7
7
8
8
8
8
7
7
8
7
7
7
8
8
7
8
8
7
8
6
7
4
7
4
8
6
6
5
7
7
3
6
6
6
4
8
7
7
8
8
8
7
7
7
7
7
6
7
8
7
8
8
7
7
7
7
7
8
7
8
8
7
8
5
7
4
7
7
5
6
5
6
7
7
2
5
6
6
3
7
7
7
7
7
7
7
7
7
7
6
6
6
7
7
7
7
7
6
7
7
6
7
7
7
8
8
7
8
6
7
4
7
4
8
6
6
5
6
6
7
7
7
7
3
6
6
6
4
7
7
6
6
7
7
7
7
7
6
7
7
6
5
6
8
7
8
8
7
7
7
6
7
7
7
8
7
7
7
7
8
7
8
7
8
7
7
7
7
7
7
8
7
8
7
8
7
7
8
7
8
7
7
7
7
7
8
8
7
8
8
7
6
7
8
8
8
8
7
7
8
7
8
7
7
8
8
8
8
7
7
6
7
8
8
8
8
7
7
7
7
8
8
7
8
8
8
8
8
8
8
7
7
8
7
7
7
7
8
8
7
7
7
8
7
7
7
8
7
8
7
7
8
7
8
7
7
7
8
7
7
7
7
3
7
5
7
6
6
5
6
6
6
7
7
7
7
7
4
6
6
6
3
7
7
7
6
6
6
7
7
7
7
7
7
7
6
6
7
7
7
8
7
5
7
7
7
7
7
7
7
7
7
7
7
6
8
8
8
8
7
7
7
7
7
8
8
8
8
8
9
8
8
8
9
8
8
8
8
8
8
7
8
8
8
8
8
8
7
7
8
8
8
7
7
8
8
7
8
8
8
8


C:\Users\admin\AppData\Local\Temp\ipykernel_11892\2236847013.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  traff_temp_p['std_no']=traff_std_no


,date,region_id,daily,month,traff_avg,traff_diff,traff_std,std_no
2464,2021-08-11,2301064.0,2847.0,8,22503.0,-19656.0,-0.700576,7
2465,2021-08-11,2301066.0,2601.0,8,2898.0,-297.0,0.064259,7
2466,2021-08-11,2304052.0,195342.0,8,226639.0,-31297.0,-1.160488,7
2467,2021-08-11,2304053.0,18745.0,8,17714.0,1031.0,0.116726,8
2468,2021-08-11,2304054.0,152212.0,8,131428.0,20784.0,0.897127,8


In [28]:
traff_temp_p.to_csv('data/인천교통량_변화량_8-10월.csv',index=False)